In [ ]:
import os

DATA_DIR = os.getenv('HYPERNET_DATA_DIR', os.path.join('..', '..', 'hypernet-data'))
RESULTS_DIR = os.path.join(os.getenv('HYPERNET_RESULTS_DIR', os.path.join('..', '..', 'hypernet-data', 'results')), 'grids_validation')

In [ ]:
from random import randint
from matplotlib import pyplot as plt
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from utils import load_patches, combine_patches
from python_research.experiments.utils.datasets.hyperspectral_dataset import HyperspectralDataset
from python_research.experiments.utils.datasets.subset import BalancedSubset
from python_research.experiments.multiple_feature_learning.builders.keras_builders import build_3d_model, build_settings_for_dataset, build_1d_model
from python_research.preprocessing.grids.grid_extraction import extract_grids
%matplotlib inline

PATCHES_DIRECTORY = ""
DATASET_PATH = os.path.join(DATA_DIR, 'PaviaU_corrected.npy')
DATASET_GT_PATH = os.path.join(DATA_DIR, 'PaviaU_gt.npy')
OUTPUT_PATH = RESULTS_DIR
PATCH_SIZE = (17, 30)
PIXEL_neighborhood = 1
TOTAL_NUMBER_OF_TRAIN_SAMPLES = 2700
CLASSES_COUNT = 9
PATIENCE = 15
EPOCHS = 200
BATCH_SIZE = 64
os.makedirs("grids_validation", exist_ok=True)


# Load data if path to the folder with patches is specified
if PATCHES_DIRECTORY != "":
    train_data, test_data = load_patches(PATCHES_DIRECTORY, PIXEL_neighborhood)
    dataset_image = test_data.x[:, :, randint(0, test_data.x.shape[-1])]
    train_data.normalize_labels()
    test_data.normalize_labels()
    bands_count = test_data.shape[-1]
    if PIXEL_neighborhood == 1:
        train_data.expand_dims(axis=-1)
        test_data.expand_dims(axis=-1)
    val_data = BalancedSubset(train_data, 0.1)
# Extract grids from provided dataset if path was not specified
else:
    patches, test_set, dataset_image = extract_grids(DATASET_PATH, DATASET_GT_PATH, PATCH_SIZE, 
                                                     TOTAL_NUMBER_OF_TRAIN_SAMPLES)
    train_data, test_data = combine_patches(patches[0], patches[1], test_set[0], test_set[1], 
                                            PIXEL_neighborhood)
    train_data.normalize_labels()
    test_data.normalize_labels()
    if PIXEL_neighborhood == 1:
        train_data.expand_dims(axis=-1)
        test_data.expand_dims(axis=-1)
    val_data = BalancedSubset(train_data, 0.1)
# Show location of extracted grids
plt.imshow(dataset_image)
plt.show()


In [ ]:

# Normalize data
max_ = train_data.max if train_data.max > val_data.max else val_data.max
min_ = train_data.min if train_data.min < val_data.min else val_data.min
train_data.normalize_min_max(min_=min_, max_=max_)
val_data.normalize_min_max(min_=min_, max_=max_)
test_data.normalize_min_max(min_=min_, max_=max_)

# Build model
if PIXEL_neighborhood == 1:
    model = build_1d_model((test_data.shape[1:]), 200, 5, CLASSES_COUNT)
else:
    settings = build_settings_for_dataset((PIXEL_neighborhood,
                                           PIXEL_neighborhood))
    model = build_3d_model(settings, CLASSES_COUNT, test_data.shape[-1])
print(model.summary())


In [ ]:
# Callbacks
early = EarlyStopping(patience=PATIENCE)
checkpoint = ModelCheckpoint(OUTPUT_PATH + "_model", save_best_only=True)

model.fit(x=train_data.get_data(), y=train_data.get_one_hot_labels(CLASSES_COUNT), batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=False,
          callbacks=[early, checkpoint], validation_data=[val_data.get_data(), val_data.get_one_hot_labels(CLASSES_COUNT)])

best_model = load_model(OUTPUT_PATH + "_model")

# Evaluate test set score
accuracy = best_model.evaluate(x=test_data.get_data(), y=test_data.get_one_hot_labels(CLASSES_COUNT))[1]
print("Test set accuracy: {}".format(accuracy))
